In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
import nibabel as nib
from skimage.measure import label

In [2]:
def normalize(volume):
    # Normalize into range [0, 1]
    vol_max = volume.max()
    vol_min = volume.min()
    volume = (volume - vol_min) / (vol_max - vol_min)
    return volume

def morphology(seg_vol, kernel_size = (30, 30)):
    (z, h, w) = seg_vol.shape
    kernel1 =  cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size)
    kernel2 =  cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size)
    for i in range(z):
        axial = cv2.morphologyEx(seg_vol[i,:,:], cv2.MORPH_CLOSE, kernel1)
        seg_vol[i,:,:] = cv2.morphologyEx(axial, cv2.MORPH_OPEN, kernel2)
    for i in range(h):
        coronal = cv2.morphologyEx(seg_vol[:,i,:], cv2.MORPH_CLOSE, kernel1)
        seg_vol[:,i,:] = cv2.morphologyEx(coronal, cv2.MORPH_OPEN, kernel2)
    """for i in range(w):
        sagittal = cv2.morphologyEx(seg_vol[:,:,i], cv2.MORPH_CLOSE, kernel1)
        seg_vol[:,:,i] = cv2.morphologyEx(sagittal, cv2.MORPH_OPEN, kernel2)"""
    return seg_vol

def get_CC_largerThanTh(arr, thresh=8000,dbg=False):
    if dbg:
        dbg_CC(arr, prec=0.02)

    print('Applying Connected Component and take components with num pixels > max_pixels')
    labels = label(arr)
    print('Found ', labels.max(), 'labels')
    max_label = 0
    # Find largestCC
    large_labels = []
    for c_label in range(1, labels.max()+1):
        curr_num_bins = np.sum(np.where(labels == c_label, 1, 0))
        print(c_label, ':', curr_num_bins)
        if curr_num_bins > thresh:
            large_labels.append(c_label)
    print('Max CC label is: ', max_label)

    print('Num liver before CC: ', np.sum(arr))
    is_first = True
    for c_label in large_labels:
        if is_first:
            arr = np.where(labels == c_label, 1, 0)
            is_first = False
        else:
            arr[labels == c_label] = 1
    print('Num liver After CC: ',np.sum(arr) )

    if dbg:
        dbg_CC(arr,prec=0.02)
    return arr

def dbg_CC(arr, prec=0.01):
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    pos = np.where(arr == 1)
    num_points = int(np.round(prec * len(pos[0])))
    indices = np.random.permutation(len(pos[0]))[0:num_points]

    ax.scatter(pos[0][indices], pos[1][indices], pos[2][indices])
    ax.view_init(elev=230., azim=360)
    plt.show()
    plt.ioff()
    # plt.waitforbuttonpress()
    plt.close()

def get_crop_coordinates(img, pad_size=2):
    """ input: binaty @D image
        output: crop coordinates of minimal "1" area with gap padding"""
    im_h, im_w = img.shape
    liver_m, liver_n = np.where(img >= 1)
    h_min = min(liver_m) - pad_size
    h_max = max(liver_m) + pad_size
    h = h_max - h_min + 1
    w_min = min(liver_n) - pad_size
    w_max = max(liver_n) + pad_size
    w = w_max - w_min + 1
    gap = abs(h - w)
    pad_l = int(np.ceil(gap / 2.))
    pad_r = int(np.floor(gap / 2.))
    if h > w:
        w_min -= pad_l
        w_max += pad_r
        if w_min < 0:
            w_min = 0
            w_max += (0 - w_min)
        if w_max > im_w:
            w_min -= w_max - im_w
            w_max = im_w
    if h < w:
        h_min -= pad_l
        h_max += pad_r
        if h_min < 0:
            h_min = 0
            h_max += (0 - h_min)
        if h_max > im_h:
            h_min -= h_max - im_h
            h_max = im_h

    return h_min, h_max, w_min, w_max

def get_crop_coordinates_3D(img_arr, pad_size=1,dbg=False):
    """ input: binaty 3D image
        output: global crop coordinates of minimal "1" area with gap padding"""
    im_d, im_h, im_w = img_arr.shape
    liver_z, liver_h, liver_w = np.where(img_arr >= 1)
    z_min = min(liver_z) - pad_size
    z_max = max(liver_z) + pad_size
    z = z_max - z_min + 1    
    h_min = min(liver_h) - pad_size
    h_max = max(liver_h) + pad_size
    h = h_max - h_min + 1
    w_min = min(liver_w) - pad_size
    w_max = max(liver_w) + pad_size
    w = w_max - w_min + 1
    gap = abs(h - w)
    pad_l = int(np.ceil(gap / 2.))
    pad_r = int(np.floor(gap / 2.))
    if h > w:
        w_min -= pad_l
        w_max += pad_r
        if w_min < 0:
            w_min = 0
            w_max += (0 - w_min)
        if w_max > im_w:
            w_min -= w_max - im_w
            w_max = im_w
    if h < w:
        h_min -= pad_l
        h_max += pad_r
        if h_min < 0:
            h_min = 0
            h_max += (0 - h_min)
        if h_max > im_h:
            h_min -= h_max - im_h
            h_max = im_h
    if dbg:
        from mpl_toolkits.mplot3d import Axes3D
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        num_points = int(np.round(0.02 * len(liver_z)))
        indices = np.random.permutation(len(liver_z))[0:num_points]
        # ax.scatter(liver_h, liver_w, liver_z)
        ax.scatter(liver_h[indices], liver_w[indices],liver_z[indices],s=0.8)
        ax.plot([h_min,h_max,h_max,h_min,h_min],[w_min, w_min,w_max, w_max,w_min], zs=int(im_d / 2), zdir='z',color='black')
        ax.view_init(elev=180., azim=360)
        plt.show()
        plt.ioff()
        plt.waitforbuttonpress()
        plt.close()

    return z_min, z_max, h_min, h_max, w_min, w_max

In [3]:
axial_model = load_model('D:/Study/Thesis/LiTS/models/Best models/Multi-plane/Axial/tumor-liver-crops-512x512-2405_weights_best.h5')
coronal_model = load_model("D:/Study/Thesis/LiTS/models/Best models/Multi-plane/Coronal/tumor-coronal-crops-512x512_weights_best.h5")
sagittal_model = load_model('D:/Study/Thesis/LiTS/models/Best models/Multi-plane/Sagittal/tumor-sagittal-crops-512x512_weights_best.h5')

In [4]:
test_list = ['106', '107', '108', '109', '110']
slice_list = os.listdir("D:/Study/Thesis/LiTS/datasets-png/test/ct/ct")

In [5]:
z_list = []
for id in test_list:
    count = 0
    for filename in slice_list:
        if id == (filename.split('-')[1]).split('_')[0]:
            count += 1
    z_list.append(count)

In [6]:
z_list

[771, 771, 856, 756, 816]

In [7]:
ct_path = 'D:/Study/Thesis/LiTS/datasets-png/test/ct/ct'
seg_path = "D:/Study/Thesis/LiTS/LiverSeg-UNet-ResNet50-20220528T101045Z-001/LiverSeg-UNet-ResNet50"

In [8]:
"""image_arr = np.zeros((z_list[0], 512, 512, 3)).astype('uint8')
liver_seg = np.zeros((z_list[0], 512, 512)).astype('uint8')
for i in range(z_list[0]):
    image_arr[i] = cv2.imread(os.path.join(ct_path, f'volume-{test_list[0]}_{i}.png'))
    liver_seg[i] = cv2.imread(os.path.join(seg_path, f'segmentation-{test_list[0]}_{i}.png'), cv2.IMREAD_GRAYSCALE)"""


"image_arr = np.zeros((z_list[0], 512, 512, 3)).astype('uint8')\nliver_seg = np.zeros((z_list[0], 512, 512)).astype('uint8')\nfor i in range(z_list[0]):\n    image_arr[i] = cv2.imread(os.path.join(ct_path, f'volume-{test_list[0]}_{i}.png'))\n    liver_seg[i] = cv2.imread(os.path.join(seg_path, f'segmentation-{test_list[0]}_{i}.png'), cv2.IMREAD_GRAYSCALE)"

In [9]:
"""liver_seg = liver_seg / 255
liver_seg[liver_seg >= 0.5] = 1
liver_seg[liver_seg < 0.5] = 0"""

'liver_seg = liver_seg / 255\nliver_seg[liver_seg >= 0.5] = 1\nliver_seg[liver_seg < 0.5] = 0'

In [10]:
"""plt.imshow(liver_seg[319,:,:])"""

'plt.imshow(liver_seg[319,:,:])'

In [11]:
"""(z, h, w) = liver_seg.shape
liver_seg = morphology(liver_seg)"""

'(z, h, w) = liver_seg.shape\nliver_seg = morphology(liver_seg)'

In [12]:
"""plt.imshow(liver_seg[341,:,:])"""

'plt.imshow(liver_seg[341,:,:])'

In [13]:
"""image_arr = normalize(image_arr)"""

'image_arr = normalize(image_arr)'

In [14]:
"""seg_liver_CC = get_CC_largerThanTh(np.where(liver_seg > 0, 1, 0), dbg=False)
# 3-D Crop coordinates
(z1, z2, h1, h2, w1, w2) = get_crop_coordinates_3D(seg_liver_CC, dbg=False)
seg_lesion_arr = np.zeros((z, h, w)).astype('uint8')
seg_coronal_arr = np.zeros((z, h, w)).astype('uint8')
seg_sagittal_arr = np.zeros((z, h, w)).astype('uint8')
# Crop in 3D
crop_img_arr = image_arr[z1:z2, h1:h2, w1:w2, :]
crop_mask_arr = liver_seg[z1:z2, h1:h2, w1:w2]
liver_crop_h, liver_crop_w = (512, 512)
curr_liver_crop_d, curr_liver_crop_h, curr_liver_crop_w = crop_mask_arr.shape
print(curr_liver_crop_d, curr_liver_crop_h, curr_liver_crop_w)"""

"seg_liver_CC = get_CC_largerThanTh(np.where(liver_seg > 0, 1, 0), dbg=False)\n# 3-D Crop coordinates\n(z1, z2, h1, h2, w1, w2) = get_crop_coordinates_3D(seg_liver_CC, dbg=False)\nseg_lesion_arr = np.zeros((z, h, w)).astype('uint8')\nseg_coronal_arr = np.zeros((z, h, w)).astype('uint8')\nseg_sagittal_arr = np.zeros((z, h, w)).astype('uint8')\n# Crop in 3D\ncrop_img_arr = image_arr[z1:z2, h1:h2, w1:w2, :]\ncrop_mask_arr = liver_seg[z1:z2, h1:h2, w1:w2]\nliver_crop_h, liver_crop_w = (512, 512)\ncurr_liver_crop_d, curr_liver_crop_h, curr_liver_crop_w = crop_mask_arr.shape\nprint(curr_liver_crop_d, curr_liver_crop_h, curr_liver_crop_w)"

In [15]:
"""for i in range(curr_liver_crop_d):
    if crop_mask_arr[i].sum() == 0:
        pred_tumor = np.zeros((liver_crop_h, liver_crop_w)).astype('uint8')
    else:
        crop_img = cv2.resize(crop_img_arr[i], (liver_crop_h, liver_crop_w), interpolation=cv2.INTER_CUBIC)
        crop_img = np.expand_dims(crop_img, axis=0)
        pred_tumor = axial_model.predict(crop_img, verbose=1)[0][:,:,0]
        pred_tumor[pred_tumor >= 0.5] = 1
        pred_tumor[pred_tumor < 0.5] = 0


    pred_tumor = cv2.resize(pred_tumor, (curr_liver_crop_h, curr_liver_crop_w), interpolation=cv2.INTER_NEAREST)
    pred_tumor[crop_mask_arr[i] == 0] = 0
    seg_lesion_arr[i+z1, h1:h2, w1:w2] = pred_tumor
seg_lesion_arr.max()"""

"for i in range(curr_liver_crop_d):\n    if crop_mask_arr[i].sum() == 0:\n        pred_tumor = np.zeros((liver_crop_h, liver_crop_w)).astype('uint8')\n    else:\n        crop_img = cv2.resize(crop_img_arr[i], (liver_crop_h, liver_crop_w), interpolation=cv2.INTER_CUBIC)\n        crop_img = np.expand_dims(crop_img, axis=0)\n        pred_tumor = axial_model.predict(crop_img, verbose=1)[0][:,:,0]\n        pred_tumor[pred_tumor >= 0.5] = 1\n        pred_tumor[pred_tumor < 0.5] = 0\n\n\n    pred_tumor = cv2.resize(pred_tumor, (curr_liver_crop_h, curr_liver_crop_w), interpolation=cv2.INTER_NEAREST)\n    pred_tumor[crop_mask_arr[i] == 0] = 0\n    seg_lesion_arr[i+z1, h1:h2, w1:w2] = pred_tumor\nseg_lesion_arr.max()"

In [16]:
"""for i in range(curr_liver_crop_h):
    if crop_mask_arr[:,i,:].sum() == 0:
        coronal_pred_tumor = np.zeros((liver_crop_h, liver_crop_w)).astype('uint8')
    else:
        coronal_crop_img = cv2.resize(crop_img_arr[:,i,:], (liver_crop_w, liver_crop_h), interpolation=cv2.INTER_CUBIC)
        coronal_crop_img = np.expand_dims(coronal_crop_img, axis=0)
        coronal_pred_tumor = coronal_model.predict(coronal_crop_img, verbose=1)[0][:,:,0]
        coronal_pred_tumor[coronal_pred_tumor >= 0.5] = 1
        coronal_pred_tumor[coronal_pred_tumor < 0.5] = 0


    coronal_pred_tumor = cv2.resize(coronal_pred_tumor, (curr_liver_crop_h, curr_liver_crop_d), interpolation=cv2.INTER_NEAREST)
    coronal_pred_tumor[crop_mask_arr[:,i,:] == 0] = 0
    seg_coronal_arr[z1:z2, i+h1, w1:w2] = coronal_pred_tumor
seg_coronal_arr.max()"""

"for i in range(curr_liver_crop_h):\n    if crop_mask_arr[:,i,:].sum() == 0:\n        coronal_pred_tumor = np.zeros((liver_crop_h, liver_crop_w)).astype('uint8')\n    else:\n        coronal_crop_img = cv2.resize(crop_img_arr[:,i,:], (liver_crop_w, liver_crop_h), interpolation=cv2.INTER_CUBIC)\n        coronal_crop_img = np.expand_dims(coronal_crop_img, axis=0)\n        coronal_pred_tumor = coronal_model.predict(coronal_crop_img, verbose=1)[0][:,:,0]\n        coronal_pred_tumor[coronal_pred_tumor >= 0.5] = 1\n        coronal_pred_tumor[coronal_pred_tumor < 0.5] = 0\n\n\n    coronal_pred_tumor = cv2.resize(coronal_pred_tumor, (curr_liver_crop_h, curr_liver_crop_d), interpolation=cv2.INTER_NEAREST)\n    coronal_pred_tumor[crop_mask_arr[:,i,:] == 0] = 0\n    seg_coronal_arr[z1:z2, i+h1, w1:w2] = coronal_pred_tumor\nseg_coronal_arr.max()"

In [17]:
"""for i in range(curr_liver_crop_w):
    if crop_mask_arr[:,:,i].sum() == 0:
        sagittal_pred_tumor = np.zeros((liver_crop_h, liver_crop_w)).astype('uint8')
    else:
        sagittal_crop_img = cv2.resize(crop_img_arr[:,:,i], (liver_crop_w, liver_crop_h), interpolation=cv2.INTER_CUBIC)
        sagittal_crop_img = np.expand_dims(sagittal_crop_img, axis=0)
        sagittal_pred_tumor = sagittal_model.predict(sagittal_crop_img, verbose=1)[0][:,:,0]
        sagittal_pred_tumor[sagittal_pred_tumor >= 0.5] = 1
        sagittal_pred_tumor[sagittal_pred_tumor < 0.5] = 0


    sagittal_pred_tumor = cv2.resize(sagittal_pred_tumor, (curr_liver_crop_w, curr_liver_crop_d), interpolation=cv2.INTER_NEAREST)
    sagittal_pred_tumor[crop_mask_arr[:,:,i] == 0] = 0
    seg_sagittal_arr[z1:z2, h1:h2, i+w1] = sagittal_pred_tumor
seg_sagittal_arr.max()"""

"for i in range(curr_liver_crop_w):\n    if crop_mask_arr[:,:,i].sum() == 0:\n        sagittal_pred_tumor = np.zeros((liver_crop_h, liver_crop_w)).astype('uint8')\n    else:\n        sagittal_crop_img = cv2.resize(crop_img_arr[:,:,i], (liver_crop_w, liver_crop_h), interpolation=cv2.INTER_CUBIC)\n        sagittal_crop_img = np.expand_dims(sagittal_crop_img, axis=0)\n        sagittal_pred_tumor = sagittal_model.predict(sagittal_crop_img, verbose=1)[0][:,:,0]\n        sagittal_pred_tumor[sagittal_pred_tumor >= 0.5] = 1\n        sagittal_pred_tumor[sagittal_pred_tumor < 0.5] = 0\n\n\n    sagittal_pred_tumor = cv2.resize(sagittal_pred_tumor, (curr_liver_crop_w, curr_liver_crop_d), interpolation=cv2.INTER_NEAREST)\n    sagittal_pred_tumor[crop_mask_arr[:,:,i] == 0] = 0\n    seg_sagittal_arr[z1:z2, h1:h2, i+w1] = sagittal_pred_tumor\nseg_sagittal_arr.max()"

In [18]:
"""liver_seg[seg_lesion_arr == 1] = 2
liver_seg[seg_coronal_arr == 1] = 2
liver_seg[seg_sagittal_arr == 1] = 2"""

'liver_seg[seg_lesion_arr == 1] = 2\nliver_seg[seg_coronal_arr == 1] = 2\nliver_seg[seg_sagittal_arr == 1] = 2'

In [19]:
"""liver_seg[liver_seg == 1] = 0"""

'liver_seg[liver_seg == 1] = 0'

In [20]:
"""plt.imshow(liver_seg[500], cmap='gray')"""

"plt.imshow(liver_seg[500], cmap='gray')"

In [21]:
"""os.mkdir('pred_mask')"""

"os.mkdir('pred_mask')"

In [22]:
save_path = 'D:/Study/Thesis/LiTS/Final-result/TumorSeg'

In [23]:
for i in range(len(test_list)):
    image_arr = np.zeros((z_list[i], 512, 512, 3)).astype('uint8')
    liver_seg = np.zeros((z_list[i], 512, 512)).astype('uint8')
    for j in range(z_list[i]):
        image_arr[j] = cv2.imread(os.path.join(ct_path, f'volume-{test_list[i]}_{j}.png'))
        liver_seg[j] = cv2.imread(os.path.join(seg_path, f'segmentation-{test_list[i]}_{j}.png'), cv2.IMREAD_GRAYSCALE)
    
    liver_seg = liver_seg / 255
    liver_seg[liver_seg >= 0.5] = 1
    liver_seg[liver_seg < 0.5] = 0
    (z, h, w) = liver_seg.shape
    liver_seg = morphology(liver_seg)
    image_arr = normalize(image_arr)
    
    seg_liver_CC = get_CC_largerThanTh(np.where(liver_seg > 0, 1, 0), dbg=False)
    # 3-D Crop coordinates
    (z1, z2, h1, h2, w1, w2) = get_crop_coordinates_3D(seg_liver_CC, dbg=False)
    seg_lesion_arr = np.zeros((z, h, w)).astype('uint8')
    """seg_coronal_arr = np.zeros((z, h, w)).astype('uint8')
    seg_sagittal_arr = np.zeros((z, h, w)).astype('uint8')"""
    # Crop in 3D
    crop_img_arr = image_arr[z1:z2, h1:h2, w1:w2, :]
    crop_mask_arr = liver_seg[z1:z2, h1:h2, w1:w2]
    liver_crop_h, liver_crop_w = (512, 512)
    curr_liver_crop_d, curr_liver_crop_h, curr_liver_crop_w = crop_mask_arr.shape
    
    for j in range(curr_liver_crop_d):
        if crop_mask_arr[j].sum() == 0:
            pred_tumor = np.zeros((liver_crop_h, liver_crop_w)).astype('uint8')
        else:
            crop_img = cv2.resize(crop_img_arr[j], (liver_crop_h, liver_crop_w), interpolation=cv2.INTER_CUBIC)
            crop_img = np.expand_dims(crop_img, axis=0)
            pred_tumor = axial_model.predict(crop_img, verbose=1)[0][:,:,0]
            pred_tumor[pred_tumor >= 0.5] = 1
            pred_tumor[pred_tumor < 0.5] = 0


        pred_tumor = cv2.resize(pred_tumor, (curr_liver_crop_h, curr_liver_crop_w), interpolation=cv2.INTER_NEAREST)
        pred_tumor[crop_mask_arr[j] == 0] = 0
        seg_lesion_arr[j+z1, h1:h2, w1:w2] = pred_tumor
        
    """for j in range(curr_liver_crop_h):
        if crop_mask_arr[:,j,:].sum() == 0:
            coronal_pred_tumor = np.zeros((liver_crop_h, liver_crop_w)).astype('uint8')
        else:
            coronal_crop_img = cv2.resize(crop_img_arr[:,j,:], (liver_crop_w, liver_crop_h), interpolation=cv2.INTER_CUBIC)
            coronal_crop_img = np.expand_dims(coronal_crop_img, axis=0)
            coronal_pred_tumor = coronal_model.predict(coronal_crop_img, verbose=1)[0][:,:,0]
            coronal_pred_tumor[coronal_pred_tumor >= 0.5] = 1
            coronal_pred_tumor[coronal_pred_tumor < 0.5] = 0


        coronal_pred_tumor = cv2.resize(coronal_pred_tumor, (curr_liver_crop_h, curr_liver_crop_d), interpolation=cv2.INTER_NEAREST)
        coronal_pred_tumor[crop_mask_arr[:,j,:] == 0] = 0
        seg_coronal_arr[z1:z2, j+h1, w1:w2] = coronal_pred_tumor
    
    for j in range(curr_liver_crop_w):
        if crop_mask_arr[:,:,j].sum() == 0:
            sagittal_pred_tumor = np.zeros((liver_crop_h, liver_crop_w)).astype('uint8')
        else:
            sagittal_crop_img = cv2.resize(crop_img_arr[:,:,j], (liver_crop_w, liver_crop_h), interpolation=cv2.INTER_CUBIC)
            sagittal_crop_img = np.expand_dims(sagittal_crop_img, axis=0)
            sagittal_pred_tumor = sagittal_model.predict(sagittal_crop_img, verbose=1)[0][:,:,0]
            sagittal_pred_tumor[sagittal_pred_tumor >= 0.5] = 1
            sagittal_pred_tumor[sagittal_pred_tumor < 0.5] = 0

        sagittal_pred_tumor = cv2.resize(sagittal_pred_tumor, (curr_liver_crop_w, curr_liver_crop_d), interpolation=cv2.INTER_NEAREST)
        sagittal_pred_tumor[crop_mask_arr[:,:,j] == 0] = 0
        seg_sagittal_arr[z1:z2, h1:h2, j+w1] = sagittal_pred_tumor"""
    
    liver_seg[seg_lesion_arr == 1] = 2
    """liver_seg[seg_coronal_arr == 1] = 2
    liver_seg[seg_sagittal_arr == 1] = 2"""
    liver_seg[liver_seg == 1] = 0
    liver_seg[liver_seg == 2] = 1
    liver_seg = liver_seg * 255
    
    for j in range(z):
        cv2.imwrite(os.path.join(save_path, f'segmentation-{test_list[i]}_{j}.png'), liver_seg[j])

Applying Connected Component and take components with num pixels > max_pixels
Found  1 labels
1 : 2287222
Max CC label is:  0
Num liver before CC:  2287222
Num liver After CC:  2287222
1/1 [==============================] - 1s 1s/step
Applying Connected Component and take components with num pixels > max_pixels
Found  1 labels
1 : 2896491
Max CC label is:  0
Num liver before CC:  2896491
Num liver After CC:  2896491
1/1 [==============================] - 1s 1s/step
Applying Connected Component and take components with num pixels > max_pixels
Found  2 labels
1 : 1579319
2 : 4812552
Max CC label is:  0
Num liver before CC:  6391871
Num liver After CC:  6391871
1/1 [==============================] - 1s 1s/step
Applying Connected Component and take components with num pixels > max_pixels
Found  1 labels
1 : 3140793
Max CC label is:  0
Num liver before CC:  3140793
Num liver After CC:  3140793
1/1 [==============================] - 1s 1s/step
Applying Connected Component and take components

In [24]:
def dice_coef(im1, im2, empty_score=1.0):

    im1 = np.asarray(im1).astype(bool)
    im2 = np.asarray(im2).astype(bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    im_sum = im1.sum() + im2.sum()
    if im_sum == 0:
        return empty_score

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return (2. * intersection.sum() / im_sum)
def IoU(im1, im2, empty_score = 1.0):
    im1 = np.asarray(im1).astype(bool)
    im2 = np.asarray(im2).astype(bool)
    intersection = np.logical_and(im1, im2)
    union = np.logical_or(im1, im2)
    if np.sum(union) == 0:
        return empty_score 
    return (np.sum(intersection) / np.sum(union))

In [30]:
save_path = 'D:/Study/Thesis/LiTS/Final-result/Polar'


In [31]:
dice_test = []
iou_test = []
mask_path = 'D:/Study/Thesis/LiTS/datasets-png/test/tumor_seg/tumor_seg'
liver_path = 'D:/Study/Thesis/LiTS/datasets-png/test/liver_seg/liver_seg'
mask_list = os.listdir(save_path)
print(len(mask_list))
for filename in mask_list:
    true_mask = cv2.imread(os.path.join(mask_path, filename), cv2.IMREAD_GRAYSCALE)
    pred_mask = cv2.imread(os.path.join(save_path, filename), cv2.IMREAD_GRAYSCALE)
    liver_mask = cv2.imread(os.path.join(liver_path, filename), cv2.IMREAD_GRAYSCALE)
    if liver_mask.sum() != 0:
        dice_test.append(dice_coef(true_mask, pred_mask))
        iou_test.append(IoU(true_mask, pred_mask))


print('Dice Mean:', np.array(dice_test).mean())
print('IoU Mean:', np.array(iou_test).mean())

7780
Dice Mean: 0.7271510670441188
IoU Mean: 0.6835057416261797
